<font color="orange"><u><h1>PHASE 1: AWS S3 Preparatory Work:</h1></u></font>
<ul>
    <li><b>Connect to AWS S3<b></li>
    <li><b>Create a bucket in S3 to house our original images and their blurred counterparts</b></li>
    <li><b>Upload sample images (mix of jpeg and png formats which are the only 2 formats that AWS Rekognition can work with)</b></li>
</ul>

In [22]:
# Connect to AWS S3 in London
import boto3

s3 = boto3.client('s3',region_name = 'eu-west-2') #London

In [197]:
# Create 'videre-oculo' bucket
from pprint import pprint  # Dictionary response is better displayed...

try:
    response = s3.create_bucket(Bucket='videre-oculo', CreateBucketConfiguration={'LocationConstraint': 'eu-west-2'})
    pprint(response)
except Exception as e:
    print('Exception:', e)

{'Location': 'http://videre-oculo.s3.amazonaws.com/',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
                                      'date': 'Sun, 19 Jul 2020 08:27:31 GMT',
                                      'location': 'http://videre-oculo.s3.amazonaws.com/',
                                      'server': 'AmazonS3',
                                      'x-amz-id-2': '7ZN2YmbXsVXZlh6rkCKHP3La+TKNs6jTIsl7W7F7OlKPshfGnduwYR7ou1TwPdpzplD+DCX5kQs=',
                                      'x-amz-request-id': '1Z3XEWBPAZBK1K7T'},
                      'HTTPStatusCode': 200,
                      'HostId': '7ZN2YmbXsVXZlh6rkCKHP3La+TKNs6jTIsl7W7F7OlKPshfGnduwYR7ou1TwPdpzplD+DCX5kQs=',
                      'RequestId': '1Z3XEWBPAZBK1K7T',
                      'RetryAttempts': 0}}


<hr>
<h2>Upload local files to bucket</h2>

<b><u>Note:</u> Not strictly required since I'll be passing my local images to Rekognition (rather than images in an S3 bucket) but shows how I would do it.<br> Also, I'm only uploading a handful of jpeg and png images for the sake of this demonstration.</b>

In [23]:
# Upload local images to new bucket (all within the 5MB limit that Rekognition imposes)
import os

local_image_folder = '/Users/fredericgugic/Oculo/Faces'
try:
    for file in os.listdir(local_image_folder):
        # Ignore the folder's .DS_Store file. Don't need that!?!
        if file.startswith('.'):
            continue
        # Get file extension (will end up either jpeg/jpg or png)
        ext = file.split('.')[-1]
        # Upload local file to bucket
        s3.upload_file(Filename=os.path.join(local_image_folder,file), Bucket='videre-oculo',
                       Key=f'originals/{file}',
                       ExtraArgs={'ContentType':f'image/{ext}'})
        print('Uploaded', file)
except Exception as e:
    print('Exception:', e)

Uploaded Les2Alpes.jpg
Uploaded Hat.jpg
Uploaded Cinema.png
Uploaded ChristmasGames.jpg
Uploaded PlayTime.png
Uploaded Holidays.png
Uploaded Cooking.png
Uploaded Shopping.png
Uploaded IceRink.jpg
Uploaded Chatting.jpg
Uploaded SatOnSofa.jpg


<hr>
<font color="orange"><u><h1>PHASE 2: More preparatory work:</h1></u></font>
<ul><li><b>Make sure all local jpeg/jpg images are stored locally with the right orientation.</li></ul>
This is to ensure that the 'mapping' between bounding box ratios/coordinates returned by Rekognition and actual source image is the same so that blurring can be applied to the right sections of an image.</b>

In [31]:
# Use PIL to get access to the EXIF orientation details of jpeg images 
# so that they can be correctly oriented before the face detection process
from PIL import Image

# Dictionary of orientations and corresponding correction to help restore the proper orientation
actions = {2: Image.FLIP_TOP_BOTTOM, 3: Image.ROTATE_180, 4: Image.ROTATE_180, 5: Image.ROTATE_270, 
           6: Image.ROTATE_270,      7: Image.ROTATE_90,  8: Image.ROTATE_90}

# Go through local images 
for file in os.listdir(local_image_folder):
    try:
        # Get file extension
        ext = file.split('.')[-1]
        # Only deal with jpeg/jpg files
        if ext.lower() not in ['jpeg', 'jpg']:
            continue
        file_path = os.path.join(local_image_folder,file)
        # Get local image
        local_image = Image.open(file_path)
        # Get orientation of image from EXIF details (key 274). Default to 1 when no EXIF details exist.
        orientation = dict(local_image.getexif()).get(274, 1)
        # Rotate/Flip the image as necessary
        if orientation in [2, 3, 6, 8]:
            local_image = local_image.transpose(actions[orientation])
        if orientation in [4, 5, 7]:
            local_image = local_image.transpose(actions[orientation]).transpose(Image.FLIP_LEFT_RIGHT)
        # Save correctly oriented image back to local folder
        local_image.save(f'/Users/fredericgugic/Oculo/Faces/{file}')
    except Exception as e:
        print(e)              

<hr>
<font color="orange"><u><h1>PHASE 3: AWS Face Detection With Rekognition:</h1></u></font>
<ul>
    <li><b>Connect to AWS Rekognition<b></li>
    <li><b>Loop through local images</b></li>
        <ol>
            <li>Send image to Rekognition for face detection</li>
            <li>Loop through all detected faces within an image</li>
                <ul>
                    <li>Use ratios in BoundingBox dictionary to derive actual pixel coordinates</li>
                    <li>Apply Gaussian filter to blur face within bounding box</li>
                    <li>Outline box</li>
                </ul>
            <li>Upload blurred image to 'blurred' folder in S3 bucket</li>
        </ol>
</ul>

In [24]:
# Connect to AWS Rekognition in London 
rekog = boto3.client('rekognition', region_name = 'eu-west-2')             

In [41]:
from skimage import io, filters

# Go through local images and carry out blurring process 
for file in os.listdir(local_image_folder):
    try:
        # Ignore the folder's .DS_Store file!!
        if file.startswith('.'):
            continue
        file_path = os.path.join(local_image_folder,file)
        # Get file extension
        ext = file.split('.')[-1]
        
        # Store image as ndarray in preparation for processing sections where faces are detected
        if ext.lower() in ['jpeg', 'jpg']:  # No Alpha component for jpeg/jpg, only keep RGB components
            work_image = io.imread(file_path)[:,:,:3]  # RGB
        else:
            work_image = io.imread(file_path)  # RGBA for png images
        # Get image size in pixels(height H and width W)
        H, W = work_image.shape[:2]
 
        # Let AWS Rekognition detect faces in our local image 
        with open(file_path, 'rb') as the_image:
            faces = rekog.detect_faces(Image={'Bytes': the_image.read()})
        
        # Fit each blurred face within a white 1px outline rectangle so that it stands out a bit more 
        for face in faces["FaceDetails"]:
            # Values in dict BoundingBox are ratios so have to be multiplied by H or W to get pixels back
            # X and Y are the coordinates of the top left corner of the bounding box
            X = int(face["BoundingBox"]["Left"] * W)
            Y = int(face["BoundingBox"]["Top"] * H)
            # Similarly, derive the width and height of the bounding box
            box_width = int(face["BoundingBox"]["Width"] * W)
            box_height = int(face["BoundingBox"]["Height"] * H)
            # Taking into account the outline rectangle we want to achieve, here's what we do:
            # - Adjust the bounding box boundaries to get a 'framed' copy of the detected face
            detected_face = work_image[Y+1:Y+box_height-1, X+1:X+box_width-1,:3]
            # - Use a Gaussian filter to blur the face
            #  (get more 'responsive' sigma value depending on the ratio of areas of whole image to bounding box)
            dynamic_sigma = 15 + (H * W) / (box_height * box_width * 2)
            blurred_face = filters.gaussian(detected_face, multichannel=True, sigma=dynamic_sigma) * 256
            # - Fill the whole of the bounding box white
            work_image[Y:Y+box_height, X:X+box_width,:3] = 255
            # - Overlay blurred face to white box, achieving the 'framing'
            work_image[Y+1:Y+box_height-1, X+1:X+box_width-1,:3] = blurred_face
            
        #Save blurred image locally and also upload to bucket 
        io.imsave(f'/Users/fredericgugic/Oculo/Blurred/{file}', work_image)
        s3.upload_file(Filename=f'/Users/fredericgugic/Oculo/Blurred/{file}', Bucket='videre-oculo',
                       Key=f'blurred/{file}',
                       ExtraArgs={'ContentType':f'image/{ext}'})
    except Exception as e:
        print(e)              